In [52]:
import penman


penman_string, alignment = "# ::snt Bill whistled past the house.\n# ::tokens [\"Bill\", \"whistled\", \"past\", \"the\", \"house\", \".\"]\n# ::lemmas [\"Bill\", \"whistle\", \"past\", \"the\", \"house\", \".\"]\n# ::alignments 0-1.1.1.1 1-1 2-1.2 4-1.2.1\n(w / whistle-01~e.1\n   :ARG0 (p / person\n            :name (n / name\n                     :op1 \"Bill\"~e.0))\n   :path (p2 / past~e.2\n             :op1 (h / house~e.4)))", [[0, 4], [5, 13], [14, 18], [19, 22]]


print(penman_string)

a = penman.decode(penman_string)

print(a.metadata,'\n\n')

print(a.triples, '\n\n')

print(a.edges(), '\n\n')

print(a.attributes(), '\n\n')




# ::snt Bill whistled past the house.
# ::tokens ["Bill", "whistled", "past", "the", "house", "."]
# ::lemmas ["Bill", "whistle", "past", "the", "house", "."]
# ::alignments 0-1.1.1.1 1-1 2-1.2 4-1.2.1
(w / whistle-01~e.1
   :ARG0 (p / person
            :name (n / name
                     :op1 "Bill"~e.0))
   :path (p2 / past~e.2
             :op1 (h / house~e.4)))
{'snt': 'Bill whistled past the house.', 'tokens': '["Bill", "whistled", "past", "the", "house", "."]', 'lemmas': '["Bill", "whistle", "past", "the", "house", "."]', 'alignments': '0-1.1.1.1 1-1 2-1.2 4-1.2.1'} 


[('w', ':instance', 'whistle-01'), ('w', ':ARG0', 'p'), ('p', ':instance', 'person'), ('p', ':name', 'n'), ('n', ':instance', 'name'), ('n', ':op1', '"Bill"'), ('w', ':path', 'p2'), ('p2', ':instance', 'past'), ('p2', ':op1', 'h'), ('h', ':instance', 'house')] 


[Edge(source='w', role=':ARG0', target='p'), Edge(source='p', role=':name', target='n'), Edge(source='w', role=':path', target='p2'), Edge(source='p2', 

In [43]:
import dgl
import torch

In [54]:
nodes = set()
edge_types = set()


for edge in a.edges():
    source, role, target = edge.source, edge.role, edge.target
    nodes.add(source)
    nodes.add(target)
    # v stands for vertex - not sure if we need multiple vertices in the future.
    edge_types.add(('v',role,'v'))

for attribute in a.attributes():
    source, role, target = attribute.source, attribute.role, attribute.target
    nodes.add(source)
    nodes.add(target)
    edge_types.add(('v',role,'v'))

print(nodes)
print(edge_types)


print('-------')
print(a.edges())
print('-------')
print(a.attributes())
print('-------')


{'h', 'w', 'p2', 'p', '"Bill"', 'n'}
{('v', ':op1', 'v'), ('v', ':name', 'v'), ('v', ':ARG0', 'v'), ('v', ':path', 'v')}
-------
[Edge(source='w', role=':ARG0', target='p'), Edge(source='p', role=':name', target='n'), Edge(source='w', role=':path', target='p2'), Edge(source='p2', role=':op1', target='h')]
-------
[Attribute(source='n', role=':op1', target='"Bill"')]
-------


In [58]:
edge_collection = {}

from penman import surface

for edge_type in edge_types:
    # v1, t, v2  = edge_type
    edge_collection[edge_type] = []

print(nodes)

print('--------------------------')

alignments = {}

node_to_index = dict(list(map(lambda j : (j[1],j[0]), enumerate(list(nodes)))))

for key, value in surface.alignments(a).items():
    node = key[0]
    node_id = node_to_index[node]
    alignments[node_id] = value.indices

print(node_to_index)
print(alignments)

print('--------------------------')





for edge in a.edges():
    source, role, target = edge.source, edge.role, edge.target
    edge_collection[('v',role,'v')].append((node_to_index[source], node_to_index[target]))

for attribute in a.attributes():
    source, role, target = attribute.source, attribute.role, attribute.target
    edge_collection[('v',role,'v')].append((node_to_index[source], node_to_index[target]))

dgl_graph = {}

for edge_type in edge_collection:
    u, v = list(zip(*edge_collection[edge_type]))
    u, v = list(u), list(v)
    dgl_graph[edge_type] = torch.tensor(u), torch.tensor(v)


print('\n\n', dgl_graph, '\n\n')

g = dgl.heterograph(dgl_graph)

import pickle


{'h', 'w', 'p2', 'p', '"Bill"', 'n'}
--------------------------
{'h': 0, 'w': 1, 'p2': 2, 'p': 3, '"Bill"': 4, 'n': 5}
{1: (1,), 5: (0,), 2: (2,), 0: (4,)}
--------------------------


 {('v', ':op1', 'v'): (tensor([2, 5]), tensor([0, 4])), ('v', ':name', 'v'): (tensor([3]), tensor([5])), ('v', ':ARG0', 'v'): (tensor([1]), tensor([3])), ('v', ':path', 'v'): (tensor([1]), tensor([2]))} 




In [ ]:

i = 100

graphs = stog.parse_sents([dataset['train'][i]['premise'], dataset['train'][i]['hypothesis']])

for graph in graphs:
    print(graph)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("hans")

In [9]:
from datasets import load_dataset

dataset = load_dataset("hans")

Reusing dataset hans (/home/david/.cache/huggingface/datasets/hans/plain_text/1.0.0/452e93cf5383f5ae39088254215b517d0da98ccaaf0af8f7ab04d8f23f67dbd9)


  0%|          | 0/2 [00:00<?, ?it/s]